In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [2]:
import pandas as pd
import numpy as np
import torch

import src.preprocessing as pre
import plotly.graph_objects as go
import src.utils as utils
from torch.utils.data import DataLoader
import src.table as table

In [3]:
train, test, mappings, meta = pre.separate_year('../data/meta.json','../data/y.csv','../data/mappings.json')

In [4]:
model = utils.create_model(train,num_epochs=100)

Epoch [1/200], Loss: 4387507.3097
Epoch [2/200], Loss: 3815341.8338
Epoch [3/200], Loss: 3168647.9581
Epoch [4/200], Loss: 2997148.7905
Epoch [5/200], Loss: 2864431.5518
Epoch [6/200], Loss: 2664083.2919
Epoch [7/200], Loss: 2523408.9176
Epoch [8/200], Loss: 2420477.2358
Epoch [9/200], Loss: 2364393.2010
Epoch [10/200], Loss: 2336705.9503
Epoch [11/200], Loss: 2304015.3888
Epoch [12/200], Loss: 2271609.8736
Epoch [13/200], Loss: 2252189.9616
Epoch [14/200], Loss: 2224168.1690
Epoch [15/200], Loss: 2199669.5284
Epoch [16/200], Loss: 2169363.0540
Epoch [17/200], Loss: 2148840.8402
Epoch [18/200], Loss: 2124193.4276
Epoch [19/200], Loss: 2093223.2287
Epoch [20/200], Loss: 2060686.4474
Epoch [21/200], Loss: 2029293.7656
Epoch [22/200], Loss: 1999104.6719
Epoch [23/200], Loss: 1955006.7464
Epoch [24/200], Loss: 1924342.4812
Epoch [25/200], Loss: 1898545.4020
Epoch [26/200], Loss: 1849898.6023
Epoch [27/200], Loss: 1812908.8615
Epoch [28/200], Loss: 1778243.1626
Epoch [29/200], Loss: 1745392

In [5]:
preds = utils.predict_gridded_harvest(model,test)

In [6]:
actuals = test.Y.detach().numpy()

In [7]:
production_plan = pre.decode(meta,mappings).reset_index(drop=True)

In [8]:
transplant_weeks = production_plan.WeekTransplanted.to_numpy()
year_preds, year_actuals = table.season_shift(transplant_weeks,preds,actuals)
year_preds.shape,year_actuals.shape

((139, 20, 51), (139, 51))

In [11]:
# Create a graph using year_preds and year_actuals

# Sum the data by the first dimension
summed_preds = np.sum(year_preds, axis=0)  # Shape: (20, 51)
summed_actuals = np.sum(year_actuals, axis=0)  # Shape: (51,)

# Create traces for the middle dimension of year_preds
traces = []
for i in range(summed_preds.shape[0]):  # Iterate over the 20 middle dimension
    trace = go.Scatter(
        x=np.arange(summed_preds.shape[1]),  # X-axis: 0 to 50
        y=summed_preds[i, :],  # Y-axis: summed predictions for each middle dimension
        mode='lines',
        name=f'Preds Trace {i+1}',
        visible=(i == 0)  # Only the first trace is visible initially
    )
    traces.append(trace)

# Create bars for year_actuals
actuals_trace = go.Bar(
    x=np.arange(summed_actuals.shape[0]),  # X-axis: 0 to 50
    y=summed_actuals,  # Y-axis: summed actuals
    name='Actuals',
    opacity=0.6
)

# Add the actuals trace to the list of traces
traces.append(actuals_trace) 

# Create the figure
fig = go.Figure(data=traces)

# Create steps for the slider
steps = []
for i in range(len(traces) - 1):  # Exclude the actuals trace from the slider
    step = dict(
        method='update',
        args=[{'visible': [j == i for j in range(len(traces) - 1)] + [True]}],  # Show only one preds trace and the actuals
        label=f'Preds Trace {i+1}'
    )
    steps.append(step)

# Create the slider
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Trace: "},
    pad={"t": 50},
    steps=steps
)]

# Update layout
fig.update_layout(
    title='Yearly Predictions and Actuals',
    xaxis_title='Time Step',
    yaxis_title='Values',
    barmode='overlay',
    sliders=sliders
)

# Show the figure
fig.show()
